# Agrupamento - Segmentação de Clientes

Este notebook realiza uma análise de Agrupamento de Dados sobre o dataset [Mall Customer Segmentation Data](https://www.kaggle.com/vjchoudhary7/customer-segmentation-tutorial-in-python). Um conjunto de dados que reúne características de 200 clientes que frequentam um shopping mall. Neste estudo veremos algumas visualizações de dados sobre os clientes, bem como o uso de agrupamento de dados para segmentação dos clientes, explorando os algoritmos k-Means e DBSCAN - e comparando seus resultados.   

> Conteúdo voltado para iniciantes na área de Aprendizado de Máquina e Ciência de Dados!

<a id="top"></a>

## Conteúdo

> **Nota**. Alguns códigos foram ocultados a fim de facilitar a leitura e dar destaque para os conteúdos mais importantes.

O notebook está organizado como segue:

- [Dados](#loading) - Carregamento dos dados.
- [Visualização](#visual) - Análise exploratória dos dados.
- [Agrupamento](#clustering) - Aplicação de algoritmos de Aprendizado de Máquina.
    - [k-Means](#kmeans) - Segmentação com k-Means.
    - [DBSCAN](#dbscan) - Segmentação com DBSCAN.
    - [Dados Sintéticos](#new_data) - Exploração sobre Dados Sintéticos.

<a id="loading"></a>

---

# Dados

- Carregamento dos dados.

[Voltar para o Topo](#top)

In [ ]:
# processamento de dados, algebra linear
import numpy as np 
import pandas as pd

# visualização de dados
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# impime os arquivos
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/customer-segmentation-tutorial-in-python/Mall_Customers.csv')
df.sample(3)

<a id="visual"></a>

---

# Visualização dos Dados

- Carregamento dos dados.

[Voltar para o Topo](#top)

## Estatística Descritiva

Nesta seção vemos dois métodos do `pandas.DataFrame` para visualizar características dos dados, tais como: (1) `.info()`, presença de valores nulos e os tipos dos dados; e (2) `.describe()`, amplitude, média, desvio padrão e quartiles.

In [ ]:
df.info()

In [ ]:
df.describe()

## Visualizações

Vamos visualizar a relação dos atributos "receita anual" x "score de gasto".

In [ ]:
sns.scatterplot(data=df, x="Annual Income (k$)", y="Spending Score (1-100)")
plt.show()

### Será que o comportamento muda com o sexo?

Vamos pintar as bolinhas de acordo com o sexo do cliente.

In [ ]:
sns.scatterplot(data=df, x="Annual Income (k$)", y="Spending Score (1-100)", hue="Gender", style="Gender")
plt.show()

### Será que o comportamento muda com a idade?

Vamos pintar as bolinhas de acordo com a idade do cliente.

In [ ]:
sns.scatterplot(data=df, x="Annual Income (k$)", y="Spending Score (1-100)", hue="Age", palette='crest')
plt.show()

<a id="clustering"></a>

---

# Agrupamento de Dados

Nesta seção exploramos:

- Seleção dos dados de interesse.
- k-Means
- DBSCAN
- Dados Sintéticos _(extra)_

[Voltar para o Topo](#top)

## Seleção dos Dados

In [ ]:
# removendo as colunas id, gênero e idade
X = df.drop(['CustomerID', 'Gender', 'Age'], axis = 1)
X.sample(3)

Abaixo tem _(ocultado)_ um código para normalizar os valores por coluna.

> O k-Means utiliza uma função de similaridade para computar a distância entre os pontos.   
> As funções de similaridade se benificiam de dados normalizados.   

> Para este estudo de caso, este procedimento não muda o resultado final.   

In [ ]:
# normalizando os valores por coluna
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)

<a id="kmeans"></a>

## k-Means

O algoritmo `KMeans` agrupa os dados tentando separar amostras em $k$ grupos de variância igual, minimizando a distância intercluster e maximizando a distância intracluster. Este algoritmo necessita saber o número de grupos, _i.e.,_ valor de $k$.    

Neste estudo, buscamos identificar 5 grupos de clientes do shopping, ou seja, utilizaremos o $k = 5$ para particionar os dados.

In [ ]:
# aprendizado de máquina
from sklearn.cluster import KMeans

In [ ]:
model = KMeans(n_clusters = 5)
model.fit(X)
df['Group'] = model.predict(X)
df.sample(3)

### Visualizando os Grupos

In [ ]:
sns.scatterplot(data=df, x="Annual Income (k$)", y="Spending Score (1-100)", hue='Group', style='Group', palette='tab10')
plt.show()

### Características dos Grupos

In [ ]:
val = df.drop(['CustomerID', 'Gender', 'Age'], axis = 1).groupby('Group').describe()

In [ ]:
# removendo alguns atributos
annual_income  = val['Annual Income (k$)'].drop(['std','min','25%','50%','75%','max'], axis = 1)
spending_score = val['Spending Score (1-100)'].drop(['count','std','25%','50%','75%'], axis = 1)
val = annual_income.merge(spending_score, left_on='Group', right_on='Group')
val.columns = ['Count', 'Income Mean', 'Score Mean', 'Score Min', 'Score Max']

In [ ]:
val

### Avaliação - Silhouette

[Silhouette](https://en.wikipedia.org/wiki/Silhouette_(clustering)) é uma métrica de interpretação e validação de algoritmos de agrupamento de dados, avaliando a consistência dentro dos grupos. Esta técnica avalia a distância intracluster (*i.e.,* a distância entre os elementos dentro do mesmo grupo) em relação a distância intercluster (*i.e.,* a distância entre elementos de grupos distintos). Neste caso, quanto maior (mais próximo de 1) a silhouette melhor a divisão dos grupos, e o contrário (mais próximo de 1) pior a divisão dos elementos nos grupos - baseando-se nas distâncias (premissa da métrica).

In [ ]:
# importa a métrica
from sklearn.metrics import silhouette_score

In [ ]:
silhouette_score(X, df['Group'])

**Discussão k-Means**   

k-Means conseguiu particionar os dados em 5 grupos bem visiveis.   
Nota-se que suas características podem ser descritas como segue:

> **Nota**. Os números dos grupos são aleatórios a cada vez que roda o algoritmo.   
> Por isso, abaixo são informado os grupos como letras e não números.

- Grupo A - Recebe pouco dinheiro, e tem score baixo.
- Grupo B - Recebe muito dinheiro, e tem score alto. _(interessante)_
- Grupo C - Recebe muito dinheiro, e tem score baixo.
- Grupo D - Recebe pouco dinheiro, e tem score alto.
- Grupo E - Recebe médio dinheiro, e tem score médio.

-----

<a id="dbscan"></a>

## DBSCAN

DBSCAN (Density-Based Spatial Clustering of Applications with Noise). Encontra amostras de núcleo de alta densidade e expande os clusters a partir delas. Bom para dados que contêm clusters de densidade semelhante. Existem dois parâmetros para o algoritmo, `min_samples` (número minímo de elementos para formar um grupo) e `eps` (raio de distância dos elementos), que definem o que queremos dizer quando dizemos denso.   

> Neste algoritmo, ele identificará o número ideal de grupos. Contudo, devemos informar outros parâmetros que também são difíceis de serem definidos.
> Logo, a escolha do algoritmos de Aprendizado de Máquina, depende do seu domínio sobre os dados e definição do problema.   

In [ ]:
from sklearn.cluster import DBSCAN

Parâmetros, $eps=0.1$ e $min\_samples=3$

In [ ]:
df['Group'] = DBSCAN(eps=0.1, min_samples=3).fit(X).labels_
df.sample(3)

In [ ]:
sns.scatterplot(data=df, x="Annual Income (k$)", y="Spending Score (1-100)", hue='Group', style='Group', palette='tab10')
plt.show()

Mudando um pouquinho o valor do $eps$ os resultados são totalmente diferentes.   
Parâmetros, $eps=0.09$ e $min\_samples=3$

In [ ]:
df['Group'] = DBSCAN(eps=0.09, min_samples=3).fit(X).labels_
df.sample(3)

In [ ]:
sns.scatterplot(data=df, x="Annual Income (k$)", y="Spending Score (1-100)", hue='Group', style='Group', palette='tab10')
plt.show()

### Avaliação - Silhouette

In [ ]:
silhouette_score(X, df['Group'])

**Discussão DBSCAN**   

Neste dados a densidade aparenta ser mais difícil de se trabalhar do que a partição do k-Means.   
Nota-se que a definisão do `eps` dificultou a partição dos dados.   

Mas quando que o DBSCAN se destaca ao k-Means? depende dos dados...   
Na próxima seção, veremos um conjunto de dados sintético e o resultado de ambos os modelos.

-----

<a id="new_data"></a>

## Dados Sintéticos

Vamos simular uma nova distribuição de dados, apenas para visualizar a vantagem do DBSCAN sobre o k-Means.

In [ ]:
# produzir dados sintéticos
from sklearn.datasets import make_blobs

Realiza a geração de um dataset sintético _(código ocultado)._

In [ ]:
X, Y = make_blobs(n_samples=1500, random_state=170)
transformation = [[0.60834549, -0.43667341], [-0.40887718, 0.85253229]]
X_aniso = np.dot(X, transformation)
X, Y = X_aniso[:, 0], X_aniso[:, 1]

In [ ]:
plt.scatter(X, Y)
plt.title("Dados Sintéticos")
plt.show()

### DBSCAN

In [ ]:
pred = DBSCAN(eps=0.4, min_samples=3).fit(X_aniso).labels_
plt.scatter(X, Y, c=pred)
plt.title("Dados Sintéticos - DBSCAN")
plt.show()

### k-Means

Vamos ver o desenpenho do k-Means sobre o conjunto de dados sintético?

In [ ]:
pred = KMeans(n_clusters = 3).fit(X_aniso).predict(X_aniso)
plt.scatter(X, Y, c=pred)
plt.title("Dados Sintéticos - k-Means")
plt.show()

-----

**Conclusão**   

Conhecer o comportamento dos dados é essencial, afim de escolher o melhor algoritmo de agrupamento.   

> 1. Nos dados dos consumidores do Shopping, o algoritmo k-Means apresentou bons resultados.   
> 2. Contudo nos dados sintéticos, o algoritmo DBSCAN se destacou pela habilidade de agrupar regiões densas.   

[Voltar para o Topo](#top)